# Notebook pro trénink s destilací nad datasetem CIFAR100
V tomto notebooku je trénován MobileNetV2 nad datasetem CIFAR100, jako učitelsý model je využíván finetunued ViT nad stejným datasetem. 

MobileNetV2 je používán s náhodnou inicializací, tréninkem pouze klasifikační hlavy inicializovaného (předtrénovaného nad ImageNetem) MobileNetuV2 a trénink celého modelu, taktéž inicializovaného. Tyto tři úlohy jsou trénovány bězným způsobem a také s pomocí destilace výše zmíněného modelu.  

Při destilaci je využíváno předpočítaných logitů ze sešitu precompute_logits.

## Import knihoven a definice metod

In [1]:
from transformers import Trainer, EarlyStoppingCallback, AutoModelForImageClassification, AutoImageProcessor
from torch.utils.data import ConcatDataset
import torch
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
dataset_part = base.get_dataset_part()
DATASET = "cifar100"

Inicializovaný MobileNetV2.

In [3]:
base.reset_seed()

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Provedení transformací nad datasetem.

In [5]:
transform = base.base_transforms()

train = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=None)
eval = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.EVAL, transform=None)
test = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TEST, transform=None)


### Standardní trénink náhodně inicializovaného modelu. 

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/vit-basetrain-model-transform", logging_dir=f"~/logs/{DATASET}/vit-basetrain-model-transform", lr=0.0005, weight_decay=0.008, adam_beta1=.95, epochs=30)
model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=100, ignore_mismatched_sizes=True)
processor = AutoImageProcessor.from_pretrained("timm/tiny_vit_5m_224.in1k")

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
from torchvision.transforms import ToTensor
def collate_fn(examples):
    to_tensor = ToTensor()
    pixel_values = torch.stack([to_tensor(example["pixel_values"]) for example in examples])
    labels = torch.tensor([example["labels"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    processing_class = processor,
    data_collator = collate_fn,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 5)]
)

In [34]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.029200,2.310220,0.393100,0.418164,0.393100,0.375002
2,1.900600,1.904595,0.481200,0.505944,0.481200,0.475442
3,1.470900,1.786745,0.513000,0.548163,0.513000,0.513766
4,1.158100,1.729847,0.535600,0.557527,0.535600,0.531547
5,0.928300,1.722173,0.547600,0.568670,0.547600,0.544768


The `save_pretrained` method is disabled for TimmWrapperImageProcessor. The image processor configuration is saved directly in `config.json` when `save_pretrained` is called for saving the model.


KeyboardInterrupt: 

In [ ]:
model.eval()

In [ ]:
trainer.evaluate(test)

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/vit-basetrain_token.pth")

In [35]:
transform = base.base_transforms()

train = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=transform)
eval = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.EVAL, transform=transform)
test = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TEST, transform=transform)

training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/vit-basetrain-token-transf", logging_dir=f"~/logs/{DATASET}/vit-basetrain-token-transf", lr=0.0005, weight_decay=0.008, adam_beta1=.95, epochs=30)
model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=100, ignore_mismatched_sizes=True)
processor = AutoImageProcessor.from_pretrained("timm/tiny_vit_5m_224.in1k")


Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    processing_class = processor,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 5)]
)

In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.642100,1.027111,0.703100,0.723367,0.703100,0.700154
2,0.763300,0.893031,0.741600,0.760275,0.741600,0.739986
3,0.550800,0.869524,0.749900,0.774201,0.749900,0.749924
4,0.421000,0.867980,0.760500,0.781796,0.760500,0.760626
5,0.317200,0.838674,0.775800,0.787255,0.775800,0.775314
6,0.255500,0.790946,0.787100,0.796470,0.787100,0.786925
7,0.216600,0.823603,0.791200,0.803079,0.791200,0.791609
8,0.162900,0.876584,0.788300,0.797620,0.788300,0.787960
9,0.140000,0.956399,0.779700,0.790966,0.779700,0.780496
10,0.116900,0.903787,0.788500,0.798007,0.788500,0.788618


TrainOutput(global_step=9390, training_loss=0.1716814164552318, metrics={'train_runtime': 5929.021, 'train_samples_per_second': 202.394, 'train_steps_per_second': 1.584, 'total_flos': 5.5315759693824e+18, 'train_loss': 0.1716814164552318, 'epoch': 30.0})

In [38]:
model.eval()

TimmWrapperForImageClassification(
  (timm_model): TinyVit(
    (patch_embed): PatchEmbed(
      (conv1): ConvNorm(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (act): GELU(approximate='none')
      (conv2): ConvNorm(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (stages): Sequential(
      (0): ConvLayer(
        (blocks): Sequential(
          (0): MBConv(
            (conv1): ConvNorm(
              (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): ConvNorm(
              (c

In [39]:
trainer.evaluate(test)

{'eval_loss': 0.9375255703926086,
 'eval_accuracy': 0.8402,
 'eval_precision': 0.8423099663864122,
 'eval_recall': 0.8402,
 'eval_f1': 0.8405098146032582,
 'eval_runtime': 12.3538,
 'eval_samples_per_second': 809.468,
 'eval_steps_per_second': 6.395,
 'epoch': 30.0}

## Definice destilačního tréninku

Třída, která upravuje hugging face trenéra pro destilaci znalostí. Nově pracuje s logity uloženými v datasetu.

### Trénink náhodně inicializovaného modelu s pomocí destilace znalostí

In [41]:
base.reset_seed()

In [42]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/vit-distilltrain", logging_dir=f"~/logs/{DATASET}/vit-distilltrain", remove_unused_columns=False, epochs=30, lr=0.00047, weight_decay=0, adam_beta1=.9, lambda_param=1, temp=6)
student_model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=100, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    processing_class = processor,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 5)]
)

In [44]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.298900,0.831203,0.683800,0.731836,0.683800,0.685063
2,0.600000,0.662224,0.729600,0.753612,0.729600,0.730670
3,0.453300,0.622127,0.755500,0.776564,0.755500,0.753489
4,0.368500,0.563021,0.770700,0.789720,0.770700,0.772211
5,0.310300,0.546491,0.779300,0.799878,0.779300,0.779068
6,0.262700,0.532122,0.782600,0.800717,0.782600,0.784513
7,0.232000,0.495274,0.797000,0.812961,0.797000,0.799272
8,0.203200,0.501735,0.795400,0.811297,0.795400,0.797779
9,0.185400,0.486403,0.797000,0.811036,0.797000,0.797992
10,0.164500,0.481768,0.797900,0.809001,0.797900,0.798922


KeyboardInterrupt: 

In [ ]:
student_model.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [ ]:
trainer.evaluate(test)

{'eval_loss': 1.09884512424469,
 'eval_accuracy': 0.5504,
 'eval_precision': 0.5808980316978757,
 'eval_recall': 0.5504,
 'eval_f1': 0.5546805225416259,
 'eval_runtime': 37.502,
 'eval_samples_per_second': 266.653,
 'eval_steps_per_second': 2.107,
 'epoch': 30.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/vit-distilltrain.pth")